In [ ]:
# Libraries
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Collection

## Web Scraping API for Passenger Volume by Bus Stops data: DO NOT RUN

In [ ]:
import requests
import pandas as pd

# Passenger volume: produces link to download csv
base_url = "http://datamall2.mytransport.sg/ltaodataservice"
endpoint = "/PV/Bus"
resource_url = base_url + endpoint
last_3_months = ['202402', '202401', '202312']
for month in last_3_months:
    query_params = {'Date': month}
    # Request data from the server
    res = requests.get(resource_url, headers={"AccountKey": LTA_KEY, "accept": "application/json"}, params=query_params)
    res_list = res.json()
    print(res_list[1])

  # Combining downloaded passenger volume data
df1 = pd.read_csv("../data/transport_node_bus_202402.csv")
df2 = pd.read_csv("../data/transport_node_bus_202401.csv")
df3 = pd.read_csv("../data/transport_node_bus_202312.csv")
pv_df = pd.concat([df1, df2, df3])

pv_df['PT_CODE'] = pv_df['PT_CODE'].apply(lambda x: '{:05d}'.format(x))

#save as csv file
pv_df.to_csv("pv.csv", index=False)

## Web Scraping API for Bus Stops location data: DO NOT RUN

In [ ]:
import requests
import pandas as pd

# bus stops
base_url = "http://datamall2.mytransport.sg/ltaodataservice"
endpoint_url1 = "/BusStops"
endpoint_url2 = "/BusStops?$skip=500"
endpoint_url3 = "/BusStops?$skip=1000"
endpoint_url4 = "/BusStops?$skip=1500"
endpoint_url5 = "/BusStops?$skip=2000"
endpoint_url6 = "/BusStops?$skip=2500"
endpoint_url7 = "/BusStops?$skip=3000"
endpoint_url8 = "/BusStops?$skip=3500"
endpoint_url9 = "/BusStops?$skip=4000"
endpoint_url10 = "/BusStops?$skip=4500"
endpoint_url11 = "/BusStops?$skip=5000"

endpoints = [endpoint_url1, endpoint_url2, endpoint_url3, endpoint_url4, endpoint_url5, endpoint_url6, endpoint_url7, endpoint_url8, endpoint_url9, endpoint_url10, endpoint_url11]


In [ ]:
busstops = []

for endpoint in endpoints:
    resource_url = base_url + endpoint
    # Request data from the server
    res = requests.get(resource_url, headers={"AccountKey": "7hrdcp4vQ0inFGn4IwWHQw==", "accept": "application/json"})
    res_list = res.json()
    df = pd.DataFrame(res_list['value'])
    busstops.append(df)

busstops_df = pd.concat(busstops, ignore_index=True)

print(busstops_df)

     BusStopCode           RoadName             Description  Latitude  \
0          01012        Victoria St     Hotel Grand Pacific  1.296848   
1          01013        Victoria St         St. Joseph's Ch  1.297710   
2          01019        Victoria St         Bras Basah Cplx  1.296990   
3          01029      Nth Bridge Rd            Opp Natl Lib  1.296673   
4          01039      Nth Bridge Rd              Bugis Cube  1.298208   
...          ...                ...                     ...       ...   
5101       99139  Changi Village Rd                   Blk 5  1.388195   
5102       99161          Nicoll Dr   Bef Changi Beach CP 3  1.390262   
5103       99171          Nicoll Dr       Changi Beach CP 2  1.391128   
5104       99181      Telok Paku Rd  Bef S'pore Aviation Ac  1.387754   
5105       99189      Telok Paku Rd      S'pore Aviation Ac  1.388414   

       Longitude  
0     103.852536  
1     103.853225  
2     103.853022  
3     103.854414  
4     103.855491  
...      

# Data Pre Processing

In [ ]:
#load data locally
pv_df = pd.read_csv("/content/pv.csv")
busstops_df = pd.read_csv("/content/busstops.csv")
print(pv_df.head(5))
print(pv_df.info())
print(busstops_df.head(5))
print(busstops_df.info())
# Based on the output, we know that our datasets have no Null values

# Join both df
pv_df['PT_CODE'] = pv_df['PT_CODE'].apply(lambda x: '{:05d}'.format(x))
busstops_df['BusStopCode'] = busstops_df['BusStopCode'].apply(lambda x: '{:05d}'.format(x))
merged_df = pv_df.merge(busstops_df, left_on='PT_CODE', right_on='BusStopCode', how='left')
print(merged_df.head(10))
print(merged_df.info())


  YEAR_MONTH          DAY_TYPE  TIME_PER_HOUR PT_TYPE  PT_CODE  \
0    2024-02  WEEKENDS/HOLIDAY           17.0     BUS    45379   
1    2024-02           WEEKDAY           17.0     BUS    45379   
2    2024-02           WEEKDAY           13.0     BUS    80051   
3    2024-02  WEEKENDS/HOLIDAY           13.0     BUS    80051   
4    2024-02           WEEKDAY           13.0     BUS     5319   

   TOTAL_TAP_IN_VOLUME  TOTAL_TAP_OUT_VOLUME  
0                  190                   120  
1                  442                   470  
2                 2873                  2873  
3                 1473                  1403  
4                  109                   540  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597316 entries, 0 to 597315
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   YEAR_MONTH            597316 non-null  object 
 1   DAY_TYPE              597316 non-null  object 
 2 

In [ ]:
# Dealing with Null
null_df=merged_df[merged_df.isnull().any(axis=1)]
# print(null_df)

# Upon research, PT_CODE 65139 has been removed since Dec 23' due to route admendment
# Remove rows with PT_CODE 65139 and assign results back to original df
merged_df = merged_df.drop(merged_df[merged_df['PT_CODE'] == '65139'].index)
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597276 entries, 0 to 597315
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   YEAR_MONTH            597276 non-null  object 
 1   DAY_TYPE              597276 non-null  object 
 2   TIME_PER_HOUR         597253 non-null  float64
 3   PT_TYPE               597276 non-null  object 
 4   PT_CODE               597276 non-null  object 
 5   TOTAL_TAP_IN_VOLUME   597276 non-null  int64  
 6   TOTAL_TAP_OUT_VOLUME  597276 non-null  int64  
 7   BusStopCode           597276 non-null  object 
 8   RoadName              597276 non-null  object 
 9   Description           597276 non-null  object 
 10  Latitude              597276 non-null  float64
 11  Longitude             597276 non-null  float64
dtypes: float64(3), int64(2), object(7)
memory usage: 59.2+ MB
None
